In [46]:
import luigi
import pandas as pd
import os

Création du dossier pour stocker les données nétoyer

In [ ]:
# Vérifiez si le dossier n'existe pas déjà

dossier = 'CleanData'

if not os.path.exists(dossier):
    # Créez le dossier
    os.makedirs(dossier, mode=0o777)
    print("Dossier créé avec succès.")
else:
    print("Le dossier existe déjà.")

In [47]:
class ReadDataCanton(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/canton.csv")

    def run(self):
        df = pd.read_csv(self.file_path, sep=";")
        # Faites quelque chose avec le dataframe, par exemple, l'afficher

        df.to_csv(self.output().path, index=False)

In [48]:
class ReadDataCDSP(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/output.csv")

    def run(self):
        df = pd.read_csv(self.file_path, sep=";")
        # Faites quelque chose avec le dataframe, par exemple, l'afficher

        df.to_csv(self.output().path, index=False)

In [49]:
class ReadAllData(luigi.Task):
    def requires(self):
        return [ReadDataCanton(file_path="./RawData/canton_t1_2017.csv"), ReadDataCDSP(file_path="./RawData/cdsp_presi2012t2_circ.csv")]
    
    def run(self):
        print("lancement")

In [50]:
luigi.build(
    [ReadAllData()], local_scheduler=False,
)


DEBUG: Checking if ReadAllData() is complete
c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\luigi\worker.py:419: UserWarning: Task ReadAllData() without outputs has no custom complete() method
  is_complete = task.complete()
DEBUG: Checking if ReadDataCanton(file_path=./RawData/canton_t1_2017.csv) is complete
DEBUG: Checking if ReadDataCDSP(file_path=./RawData/cdsp_presi2012t2_circ.csv) is complete
INFO: Informed scheduler that task   ReadAllData__99914b932b   has status   PENDING
INFO: Informed scheduler that task   ReadDataCDSP___RawData_cdsp_p_0f11afe572   has status   PENDING
INFO: Informed scheduler that task   ReadDataCanton___RawData_canton_95a4352c04   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 3
INFO: [pid 19872] Worker Worker(salt=3502862612, workers=1, host=DESKTOP-PBH195U, username=Marin, pid=19872) running   ReadDataCanton(file_path=./RawData/canton_

False